In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm

### Formatting Data

In [2]:
def f(row):
    if row['Adj Close'] - row['Open'] < 0: return -1
    else: return 1

In [4]:
df = pd.read_csv('data/GOOGL.csv')
df['Utility'] = df.apply(f, axis=1)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Utility
0,2004-08-19,50.050049,52.082081,48.028027,100.339996,50.220219,44659000,1
1,2004-08-20,50.555557,54.594593,50.300301,108.310005,54.209209,22834300,1
2,2004-08-23,55.430431,56.796795,54.579578,109.400002,54.754753,18256100,-1
3,2004-08-24,55.675674,55.855854,51.836838,104.870003,52.487488,15247300,-1
4,2004-08-25,52.532532,54.054054,51.991993,106.000008,53.053055,9188600,1


In [5]:
data = df.drop('Utility', 1)
data = data.drop('Date',1).values
target = df['Utility'].values

In [6]:
ti=[]
for i in data:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append(np.array(temp_list))
features=ti

labels = []
for i in target:
    if i == -1: labels.append([1, 0])
    else: labels.append([0, 1])

### LSTM

In [7]:
tf.reset_default_graph()
sess = tf.Session()
data = tf.placeholder(tf.float32, [None, 6, 1])
target = tf.placeholder(tf.float32, [None, 2])

num_hidden = 50
cell = tf.contrib.rnn.LSTMCell(num_hidden,state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
_pred = tf.argmax(prediction, 1)
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))

optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

init_op = tf.global_variables_initializer()
sess.run(init_op)
#sess.close()

/Users/jarvis/anaconda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [14]:
batch_size = 1000
print("Running Network")
no_of_batches = int(len(features)/batch_size)
epoch = 50000
results = []

for i in tqdm(range(epoch), total=epoch, desc='Epoch'):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = features[ptr:ptr+batch_size], labels[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize, {data: inp, target: out})
        results.append(sess.run(_pred, {data: inp, target: out}))

Epoch:   0%|          | 1/50000 [00:00<1:58:45,  7.02it/s]

Running Network


Epoch: 100%|██████████| 50000/50000 [1:30:13<00:00,  9.35it/s]


In [15]:
results[100000]

array([1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1,

In [16]:
sess.run(prediction, {data: inp, target: out})

array([[  6.79860357e-04,   9.99320149e-01],
       [  9.10495102e-01,   8.95049646e-02],
       [  9.67982858e-02,   9.03201759e-01],
       ..., 
       [  4.51701969e-01,   5.48298061e-01],
       [  4.33861226e-01,   5.66138744e-01],
       [  4.34459984e-01,   5.65540075e-01]], dtype=float32)

In [17]:
sess.run(error, {data: inp, target: out})

0.27599999

In [18]:
sess.run(minimize, {data: inp, target: out})

In [12]:
sess.close

<bound method BaseSession.close of <tensorflow.python.client.session.Session object at 0x117b18518>>